In [8]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Install Postgres Driver
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://archive.ubuntu.com/ubuntu focal-updates/main amd6

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.5.0.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [10]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="mm/dd/yy")


### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [19]:
from pyspark.sql.functions import count

# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg(count("customer_id").alias("customer_count"))
customers_df.show(20)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28735044|             9|
|   15607794|             1|
|   15596972|             1|
|   32371281|             2|
|   12084021|             1|
|   14462635|             1|
|   41052840|             5|
|   52068296|           105|
|   51739623|             1|
|   14544320|             1|
|   50093726|           151|
|   52925615|             1|
|   12266315|             1|
|   44917096|             1|
|   52833886|             1|
|   51854558|            57|
|   12404586|             2|
|   49850307|             8|
|   23478170|             1|
|   51088740|             1|
+-----------+--------------+
only showing top 20 rows



In [20]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()

In [21]:
products_df.show(20)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0029148510|Acts of War: Beha...|
|1844494896|Waiting For Kate ...|
|0451216954|Dark Lover (Black...|
|006056718X|Often Wrong, Neve...|
|0965852768|101 Spooktacular ...|
|141960435X|Flip Flop Spanish...|
|0471282502|Experimental Orga...|
|0756613647|Universe: The Def...|
|B0006YGCE8|Seismic risk asse...|
|207030602X|L Etranger (Folio...|
|1569711224|Aliens: Hive  (2n...|
|0756613744|Go Figure!: A Tot...|
|0891073825|The Cross: God's ...|
|1595140840|            Teach Me|
|1844491412|Led Zeppelin: The...|
|0060737980|Surrounded by Idi...|
|0486265250|Ancient Egyptian ...|
|0763627798|           Shoe Baby|
|097559950X|Natural Cures "Th...|
|0595362478| Accursed Inhabitant|
+----------+--------------------+
only showing top 20 rows



In [24]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent",to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

In [26]:
review_id_df.show(20)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RQ58W7SMO911M|   12076615|0385730586|     122662979| 2005-10-14|
|  RF6IUKMGL8SF|   12703090|0811828964|      56191234| 2005-10-14|
|R1DOSHH6AI622S|   12257412|1844161560|     253182049| 2005-10-14|
| RATOTLA3OF70O|   50732546|0373836635|     348672532| 2005-10-14|
|R1TNWRKIVHVYOV|   51964897|0262181533|     598678717| 2005-10-14|
|R2F53LI9KK9MOY|   31048862|0316769487|     862964341| 2005-10-14|
|R1KJ6MB7MRSQFF|   53000124|0805076069|     145341889| 2005-10-14|
|R2XIM9LT335WHE|   29732693|1581603681|     640542054| 2005-10-14|
|R1VE0FQQ0QTQJN|   48541186|0300108834|     915891133| 2005-10-14|
|R1VKEE2NWSWDRU|   15280864|0446531081|     880645124| 2005-10-14|
|R2UP6XSVYJBJ2H|   24209213|0976822105|     731931430| 2005-10-14|
|R21SYDQ70ILUC0|   52253037|1580085695|     586052746| 2005-10

In [27]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

In [28]:
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [29]:
# Configure settings for RDS
from getpass import getpass
mode = "append"
password = getpass('Enter database password')
jdbc_url="jdbc:postgresql://data-rds-aws.ctuznzdhe8mz.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [30]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [31]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [32]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [33]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)